In [163]:
import numpy as np
import pandas as pd

In [164]:
def read_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
        
    return content

In [165]:
def load_data():
    data = {}
    for index in range(3):
        X_train = pd.read_csv('Xtr%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        seq_train = pd.read_csv('Xtr%s.csv' % index, names='1').values
        X_test = pd.read_csv('Xte%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        seq_test = pd.read_csv('Xte%s.csv' % index, names='1').values
        labels = pd.read_csv('Ytr%s.csv' % index, names=('Id', 'Bound'), skiprows=[0], delimiter=',')

        data[index] = {
            'seq_train': seq_train,
            'X_train': X_train,
            'y_train': labels['Bound'].values,
            'seq_test': seq_test,
            'X_test': X_test,
            'ids': labels['Id'].values,
        }
        
    return data

DATA = load_data()

In [166]:
from data_manipulation import split_train_test_valid, get_precision

In [167]:
train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['seq_train'], DATA[0]['y_train'])
train_X.shape

(1200, 1)

## Kernel ridge regression 

In [168]:
from ridge_regression import get_ridge_prediction, kernel_ridge_regression

In [169]:
def linear_K(x1, x2):
    return np.matmul(x1.T, x2)

In [170]:
REG_PARAMS_SPAN = [10**i for i in range(-10, 10)]

def get_best_reg_param():
    test_precisions = []
    for reg in REG_PARAMS_SPAN:
        alpha = kernel_ridge_regressionzlinear_K, train_X, train_y, reg)
        pred = get_ridge_prediction(linear_K, train_X, test_X, alpha)
        test_precisions.append(get_precision(pred, test_y))
        
    best_reg_index = max(range(len(REG_PARAMS_SPAN)), key=lambda x: test_precisions[x])
    return REG_PARAMS_SPAN[best_reg_index]

SyntaxError: invalid syntax (<ipython-input-170-f0dc28671ecb>, line 6)

In [171]:
reg_param = get_best_reg_param()
alpha = kernel_ridge_regression(linear_K, train_X, train_y, reg_param)
pred = get_ridge_prediction(linear_K, train_X, valid_X, alpha)
get_precision(pred, valid_y)

TypeError: Object arrays are not currently supported

## Substring Kernel 

In [172]:
def momoize(func):
    memory = {}
    def momoized(*args):
        key = '-'.join('[%s]' % arg for arg in args)
        if key not in memory:
            memory[key] = func(*args)
        
        return memory[key]
    
    return momoized

In [173]:
LAMBDA = 0.1

@momoize
def B_k(LAMBDA, k, str1, str2):
    if k == 0:
        return 1
    
    len_1 = len(str1)
    len_2 = len(str2)
    if len_1 < k or len_2 < k:
        return 0
    
    sub_1 = str1[0:-1]
    sub_2 = str2[0:-1]
    
    return (LAMBDA*B_k(LAMBDA, k, sub_1, str2)
            + LAMBDA*B_k(LAMBDA, k, str1, sub_2)
            - LAMBDA**2*B_k(LAMBDA, k, sub_1, sub_2) 
            + (LAMBDA**2*B_k(LAMBDA, k-1, sub_1, sub_2) if str1[-1] == str2[-1] else 0)
           )

@momoize
def K_k(LAMBDA, k, str1, str2):
    if k == 0:
        return 1
    
    len_1 = len(str1)
    len_2 = len(str2)
    if len_1 < k or len_2 < k:
        return 0
    
    sub_1 = str1[0:-1]
    a = str1[-1]
    
    return (K_k(LAMBDA, k, sub_1, str2)
            + LAMBDA**2 * sum(B_k(LAMBDA, k-1, sub_1, str2[0:j]) for j in range(len_2) if str2[j] == a)
           )

In [174]:
K_k(0.1, 2, 'car', 'car') - 2*LAMBDA**4+LAMBDA**6 < 10**-5

True

In [175]:
K_k(0.1, 5, 
   'CAGCTTTTATCACCTTTGAGGGAAAGTCATATTAATTTAATACTGCACACACTTGTACAACAGATCTTCTTTACTATTAAAACTCAGTTTATCAAATCACA',
   'AATAACATACCCCACTCTTTCATCTCAATCAAAAATTGAAAAAGTCAAAGAATCCTGCTTTTTTGTTTTTCTCCAAGCCATTACCCCCTCTTGATCATTGC'
   )

3.1540274232800625e-09

In [176]:
pd.read_csv('Xtr0.csv', names='1').values.shape

(2000, 1)

In [180]:
for j in range(train_X.shape[0]):
    print(j)
    K_k(0.1, 2, train_X[1,0], train_X[j,0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [181]:
2000**2

4000000

In [177]:
def substring_kernel(str1, str2):
    n1 = str1.shape[0]
    n2 = str2.shape[0]
    
    K = np.zeros((n1,n2))
    for i in range(n1):
        print(i)
        for j in range(n2):
            K[i,j] = K_k(0.1, 2, str1[i,0], str2[j,0])
            
    return K

        
K = substring_kernel(train_X, train_X)

0


KeyboardInterrupt: 

In [ ]:
K

## Final results

In [10]:
def produce_results(preds, ids):
    data = [
        '%s,%s' % (id, pred) for pred, id in zip(preds, ids)
    ]
    
    data.insert(0, 'Ids,Bound')
    with open('submission.csv', 'w') as f:
        f.write('\n'.join(data))

In [14]:
def train_tune_and_pred_on_test():
    preds = []
    ids = []
    for k in range(3):
        train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[k]['X_train'], DATA[k]['y_train'])

        reg_param = get_best_reg_param()
        
        # Validation precision
        reg_param = get_best_reg_param()
        alpha = kernel_ridge_regression(linear_K, train_X, train_y, reg_param)
        pred = get_ridge_prediction(linear_K, train_X, valid_X, alpha)
        print("Dataset %s has found a parameter with validation precision %.3f" % (k, get_precision(pred, valid_y)))
        
        alpha = kernel_ridge_regression(linear_K, DATA[k]['X_train'], DATA[k]['y_train'], reg_param)
        pred = get_ridge_prediction(linear_K, DATA[k]['X_train'], DATA[k]['X_test'], alpha)
        
        preds += list(pred)

    produce_results(preds, range(3000))

In [15]:
train_tune_and_pred_on_test()

Dataset 0 has found a parameter with validation precision 0.613
Dataset 1 has found a parameter with validation precision 0.775
Dataset 2 has found a parameter with validation precision 0.570
